In [7]:
! pip install datasets rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=39ca9d056d66792d0098982cc60477f0cbad7698672811be0f7018f3b414b2b0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages 

In [26]:
import os
import string
from collections import defaultdict
import pandas as pd
from datasets import load_metric
dataset=pd.read_csv('data/train data.csv')
  # Step 1: Examining and Loading Data
def load_data(idx):


        # for filename in os.listdir(data_folder):
        #     if filename.endswith('.txt'):
        #         file_path = os.path.join(data_folder, filename)
        #         with open(file_path, 'r', encoding='utf-8') as file:
        #             job_descriptions.append(file.read())

    job_descriptions = dataset['article'][idx]

    return job_descriptions

# Step 2: Pre-Processing Data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text into words
    words = text.split()
    # Remove stopwords
    stopwords = set(["the", "and", "is", "in", "to", "with", "a", "for", "of", "on", "it", "as", "at", "by"])
    words = [word for word in words if word not in stopwords]
    return words

# Step 3: Calculate Word Frequencies
def calculate_word_frequencies(words):
    word_frequencies = defaultdict(int)
    for word in words:
        word_frequencies[word] += 1
    return word_frequencies

# Step 4: Sentence Scoring
def score_sentences(text, word_frequencies):
    sentences = text.split('.')
    sentence_scores = defaultdict(int)
    for sentence in sentences:
        for word in sentence.split():
            if word in word_frequencies:
                sentence_scores[sentence] += word_frequencies[word]
    return sentence_scores

# Step 5: Summary Generation
def generate_summary(sentence_scores, top_n=5):
    sorted_sentences = sorted(sentence_scores.items(), key=lambda item: item[1], reverse=True)
    summary = ''.join([sentence for sentence, score in sorted_sentences[:top_n]])
    return summary

# Main Function
def main(idx):
    job_descriptions = load_data(idx)
    if not job_descriptions:
        print("No job descriptions found. Exiting...")
        return

    all_text = ''.join(job_descriptions)
    words = preprocess_text(all_text)

    word_frequencies = calculate_word_frequencies(words)

    sentence_scores = score_sentences(all_text, word_frequencies)

    summary = generate_summary(sentence_scores)
    return summary
    # print("Summary:")
    # print(summary)

# Run the main function
if __name__ == "__main__":
    # Assuming data_folder is in the same directory as the script
    data_folder = './data'  # Modify this to the path of your data folder
    main(idx=0)




In [28]:
predicted_summary=[]
for id in range(len(dataset['article'])):
    predicted_summary.append(main(id))

In [32]:
rouge=load_metric('rouge')
rouge.add_batch(predictions=predicted_summary,references=dataset['highlights'])
scores=rouge.compute()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [37]:
for key in scores:
    print(f"{key}: {scores[key].high}")

rouge1: Score(precision=0.19075765710438736, recall=0.5630896019890103, fmeasure=0.27640552164050847)
rouge2: Score(precision=0.07429293292859208, recall=0.21603493870206003, fmeasure=0.10686503558039795)
rougeL: Score(precision=0.1139276722963859, recall=0.3434306519472218, fmeasure=0.16566279334641412)
rougeLsum: Score(precision=0.15704369978133953, recall=0.46254817605343307, fmeasure=0.2270284665121885)
